<a href="https://colab.research.google.com/github/dineshnanda16/Fitme_Startup/blob/main/genralcheck1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2
import numpy as np
import gradio as gr
import mediapipe as mp

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def calculate_reference_scale(image, ref_length_cm):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use edge detection to find lines in the image
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Use the Hough Line Transform to detect the line
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=30, maxLineGap=10)

    if lines is not None:
        # We assume the first detected line is the reference object
        for line in lines:
            x1, y1, x2, y2 = line[0]
            ref_length_px = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            break  # Use the first detected line as reference

        # Calculate the pixel-to-centimeter ratio
        scale = ref_length_cm / ref_length_px
        return scale
    else:
        raise ValueError("No reference line detected in the image.")

def process_image(image):
    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform pose detection
    results = pose.process(image_rgb)

    if not results.pose_landmarks:
        return None, "No human pose detected."

    # Extract key points
    landmarks = results.pose_landmarks.landmark
    height, width, _ = image.shape

    # Convert landmark coordinates to pixel values
    keypoints = {
        "left_shoulder": (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * width),
                          int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * height)),
        "right_shoulder": (int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * width),
                           int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * height)),
        "left_hip": (int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * width),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * height)),
        "right_hip": (int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * width),
                      int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * height)),
        "left_ankle": (int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x * width),
                       int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y * height)),
        "right_ankle": (int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x * width),
                        int(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y * height)),
        "left_eye": (int(landmarks[mp_pose.PoseLandmark.LEFT_EYE].x * width),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_EYE].y * height)),
        "right_eye": (int(landmarks[mp_pose.PoseLandmark.RIGHT_EYE].x * width),
                      int(landmarks[mp_pose.PoseLandmark.RIGHT_EYE].y * height)),
    }

    # Calculate the pixel-to-centimeter scale using the reference object
    scale_cm_per_px = calculate_reference_scale(image, ref_length_cm=24)

    # Calculate distances in pixels
    shoulder_width_px = np.linalg.norm(np.array(keypoints["left_shoulder"]) - np.array(keypoints["right_shoulder"]))
    hip_width_px = np.linalg.norm(np.array(keypoints["left_hip"]) - np.array(keypoints["right_hip"]))
    height_px = np.linalg.norm(np.array(keypoints["left_eye"]) - np.array(keypoints["left_ankle"]))

    # Convert measurements to cm using the scale
    measurements = {
        "height": height_px * scale_cm_per_px,
        "shoulder_width": shoulder_width_px * scale_cm_per_px,
        "hip_width": hip_width_px * scale_cm_per_px,
    }

    # Draw keypoints and lines for visualization
    for key, point in keypoints.items():
        cv2.circle(image, point, 5, (0, 255, 0), -1)
    cv2.line(image, keypoints["left_shoulder"], keypoints["right_shoulder"], (255, 0, 0), 2)
    cv2.line(image, keypoints["left_hip"], keypoints["right_hip"], (255, 0, 0), 2)
    cv2.line(image, keypoints["left_eye"], keypoints["left_ankle"], (255, 0, 0), 2)

    return image, measurements

# Create Gradio Interface
iface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="numpy", label="Upload Image"),
    outputs=[gr.Image(label="Output Image"), gr.JSON(label="Measurements (in cm)")],
    title="Body Measurement Estimation with Reference Object"
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://68a436d497ed5191a1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [2]:
!pip install opencv-python-headless mediapipe gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/7